In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape

In [2]:
class DCGAN:
	@staticmethod
	def build_generator(dim, depth, channels=1, inputDim=100, outputDim=512):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (dim, dim, depth)
		chanDim = -1
        
		# first set of FC => RELU => BN layers
		model.add(Dense(input_dim=inputDim, units=outputDim))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		# second set of FC => RELU => BN layers, this time preparing
		# the number of FC nodes to be reshaped into a volume
		model.add(Dense(dim * dim * depth))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
        
        # reshape the output of the previous layer set, upsample +
		# apply a transposed convolution, RELU, and BN
		model.add(Reshape(inputShape))
		model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
        
		# apply another upsample and transposed convolution, but
		# this time output the TANH activation
		model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("tanh"))
		# return the generator model
		return model
    
	@staticmethod
	def build_discriminator(width, height, depth, alpha=0.2):
		# initialize the model along with the input shape to be
		# "channels last"
		model = Sequential()
		inputShape = (height, width, depth)
		# first set of CONV => RELU layers
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
			input_shape=inputShape))
		model.add(LeakyReLU(alpha=alpha))
		# second set of CONV => RELU layers
		model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
		model.add(LeakyReLU(alpha=alpha))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(LeakyReLU(alpha=alpha))
		# sigmoid layer outputting a single value
		model.add(Dense(1))
		model.add(Activation("sigmoid"))
		# return the discriminator model
		return model

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse
import cv2
import os

In [4]:
NUM_EPOCHS = 50
BATCH_SIZE = 128
INIT_LR = 0.0001

In [5]:
print("[INFO] loading MNIST dataset...")
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])
# add in an extra dimension for the channel and scale the images
# into the range [-1, 1] (which is the range of the tanh
# function)
trainImages = np.expand_dims(trainImages, axis=-1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5

[INFO] loading MNIST dataset...


In [6]:
# build the generator
print("[INFO] building generator...")
gen = DCGAN.build_generator(7, 64, channels=1)
# build the discriminator
print("[INFO] building discriminator...")
disc = DCGAN.build_discriminator(28, 28, 1)
discOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building generator...
[INFO] building discriminator...


In [7]:
# build the adversarial model by first setting the discriminator to
# *not* be trainable, then combine the generator and discriminator
# together
print("[INFO] building GAN...")
disc.trainable = False
ganInput = Input(shape=(100,))
ganOutput = disc(gen(ganInput))
gan = Model(ganInput, ganOutput)
# compile the GAN
ganOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building GAN...


In [8]:
# randomly generate some benchmark noise so we can consistently
# visualize how the generative modeling is learning
print("[INFO] starting training...")
benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))
# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
	# show epoch information and compute the number of batches per
	# epoch
	print("[INFO] starting epoch {} of {}...".format(epoch + 1,
		NUM_EPOCHS))
	batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)
	# loop over the batches
	for i in range(0, batchesPerEpoch):
		# initialize an (empty) output path
		p = None
		# select the next batch of images, then randomly generate
		# noise for the generator to predict on
		imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
		noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        
		# generate images using the noise + generator model
		genImages = gen.predict(noise, verbose=0)
		# concatenate the *actual* images and the *generated* images,
		# construct class labels for the discriminator, and shuffle
		# the data
		X = np.concatenate((imageBatch, genImages))
		y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
		y = np.reshape(y, (-1,))
		(X, y) = shuffle(X, y)
		# train the discriminator on the data
		discLoss = disc.train_on_batch(X, y)
        
		# let's now train our generator via the adversarial model by
		# (1) generating random noise and (2) training the generator
		# with the discriminator weights frozen
		noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
		fakeLabels = [1] * BATCH_SIZE
		fakeLabels = np.reshape(fakeLabels, (-1,))
		ganLoss = gan.train_on_batch(noise, fakeLabels)
        
		# check to see if this is the end of an epoch, and if so,
		# initialize the output path
		if i == batchesPerEpoch - 1:
			p = ["output", "epoch_{}_output.png".format(
				str(epoch + 1).zfill(4))]
		# otherwise, check to see if we should visualize the current
		# batch for the epoch
		else:
			# create more visualizations early in the training
			# process
			if epoch < 10 and i % 25 == 0:
				p = ["output", "epoch_{}_step_{}.png".format(
					str(epoch + 1).zfill(4), str(i).zfill(5))]
			# visualizations later in the training process are less
			# interesting
			elif epoch >= 10 and i % 100 == 0:
				p = ["output", "epoch_{}_step_{}.png".format(
					str(epoch + 1).zfill(4), str(i).zfill(5))]
                
		# check to see if we should visualize the output of the
		# generator model on our benchmark data
		if p is not None:
			# show loss information
			print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
				"adversarial_loss={:.6f}".format(epoch + 1, i,
					discLoss, ganLoss))
			# make predictions on the benchmark noise, scale it back
			# to the range [0, 255], and generate the montage
			images = gen.predict(benchmarkNoise)
			images = ((images * 127.5) + 127.5).astype("uint8")
			images = np.repeat(images, 3, axis=-1)
			vis = build_montages(images, (28, 28), (16, 16))[0]
			# write the visualization to disk
			p = os.path.sep.join(p)
			cv2.imwrite(p, vis)

[INFO] starting training...
[INFO] starting epoch 1 of 50...
[INFO] Step 1_0: discriminator_loss=0.677924, adversarial_loss=0.653065
[INFO] Step 1_25: discriminator_loss=0.297281, adversarial_loss=0.117662
[INFO] Step 1_50: discriminator_loss=0.010984, adversarial_loss=0.025659
[INFO] Step 1_75: discriminator_loss=0.001444, adversarial_loss=0.006262
[INFO] Step 1_100: discriminator_loss=0.000743, adversarial_loss=0.003037
[INFO] Step 1_125: discriminator_loss=0.000535, adversarial_loss=0.002121
[INFO] Step 1_150: discriminator_loss=0.000618, adversarial_loss=0.001425
[INFO] Step 1_175: discriminator_loss=0.000734, adversarial_loss=0.001416
[INFO] Step 1_200: discriminator_loss=0.005991, adversarial_loss=0.002035
[INFO] Step 1_225: discriminator_loss=0.393411, adversarial_loss=2.105429
[INFO] Step 1_250: discriminator_loss=0.251844, adversarial_loss=1.280695
[INFO] Step 1_275: discriminator_loss=0.196350, adversarial_loss=0.321752
[INFO] Step 1_300: discriminator_loss=0.293868, adversar

[INFO] Step 5_425: discriminator_loss=0.562356, adversarial_loss=1.025241
[INFO] Step 5_450: discriminator_loss=0.584466, adversarial_loss=0.746879
[INFO] Step 5_475: discriminator_loss=0.541210, adversarial_loss=1.039078
[INFO] Step 5_500: discriminator_loss=0.550642, adversarial_loss=1.048534
[INFO] Step 5_525: discriminator_loss=0.573782, adversarial_loss=0.975919
[INFO] Step 5_545: discriminator_loss=0.543747, adversarial_loss=1.083851
[INFO] starting epoch 6 of 50...
[INFO] Step 6_0: discriminator_loss=0.549744, adversarial_loss=0.778370
[INFO] Step 6_25: discriminator_loss=0.562357, adversarial_loss=0.706670
[INFO] Step 6_50: discriminator_loss=0.585162, adversarial_loss=0.920459
[INFO] Step 6_75: discriminator_loss=0.560714, adversarial_loss=1.288753
[INFO] Step 6_100: discriminator_loss=0.538416, adversarial_loss=1.081023
[INFO] Step 6_125: discriminator_loss=0.561535, adversarial_loss=1.088322
[INFO] Step 6_150: discriminator_loss=0.645020, adversarial_loss=0.770923
[INFO] Ste

[INFO] Step 10_275: discriminator_loss=0.612349, adversarial_loss=0.735000
[INFO] Step 10_300: discriminator_loss=0.604622, adversarial_loss=1.038321
[INFO] Step 10_325: discriminator_loss=0.609841, adversarial_loss=0.815829
[INFO] Step 10_350: discriminator_loss=0.591509, adversarial_loss=1.154749
[INFO] Step 10_375: discriminator_loss=0.594396, adversarial_loss=1.105809
[INFO] Step 10_400: discriminator_loss=0.602663, adversarial_loss=0.862834
[INFO] Step 10_425: discriminator_loss=0.603169, adversarial_loss=0.819427
[INFO] Step 10_450: discriminator_loss=0.609713, adversarial_loss=0.929564
[INFO] Step 10_475: discriminator_loss=0.594033, adversarial_loss=0.619670
[INFO] Step 10_500: discriminator_loss=0.614398, adversarial_loss=0.859107
[INFO] Step 10_525: discriminator_loss=0.602576, adversarial_loss=0.946082
[INFO] Step 10_545: discriminator_loss=0.590531, adversarial_loss=0.740796
[INFO] starting epoch 11 of 50...
[INFO] Step 11_0: discriminator_loss=0.605493, adversarial_loss=0.

[INFO] Step 24_100: discriminator_loss=0.597027, adversarial_loss=1.365095
[INFO] Step 24_200: discriminator_loss=0.571188, adversarial_loss=0.938385
[INFO] Step 24_300: discriminator_loss=0.599546, adversarial_loss=0.873887
[INFO] Step 24_400: discriminator_loss=0.643905, adversarial_loss=0.764921
[INFO] Step 24_500: discriminator_loss=0.613748, adversarial_loss=0.905916
[INFO] Step 24_545: discriminator_loss=0.606943, adversarial_loss=1.008243
[INFO] starting epoch 25 of 50...
[INFO] Step 25_0: discriminator_loss=0.594090, adversarial_loss=0.741316
[INFO] Step 25_100: discriminator_loss=0.577712, adversarial_loss=0.959484
[INFO] Step 25_200: discriminator_loss=0.565499, adversarial_loss=1.023621
[INFO] Step 25_300: discriminator_loss=0.576311, adversarial_loss=0.826706
[INFO] Step 25_400: discriminator_loss=0.645218, adversarial_loss=0.955305
[INFO] Step 25_500: discriminator_loss=0.582257, adversarial_loss=0.911900
[INFO] Step 25_545: discriminator_loss=0.608346, adversarial_loss=1.

[INFO] Step 39_100: discriminator_loss=0.619795, adversarial_loss=1.223324
[INFO] Step 39_200: discriminator_loss=0.567982, adversarial_loss=1.172667
[INFO] Step 39_300: discriminator_loss=0.581595, adversarial_loss=0.814006
[INFO] Step 39_400: discriminator_loss=0.626509, adversarial_loss=0.848172
[INFO] Step 39_500: discriminator_loss=0.583936, adversarial_loss=0.933369
[INFO] Step 39_545: discriminator_loss=0.565812, adversarial_loss=1.160597
[INFO] starting epoch 40 of 50...
[INFO] Step 40_0: discriminator_loss=0.595308, adversarial_loss=0.841460
[INFO] Step 40_100: discriminator_loss=0.563227, adversarial_loss=0.740018
[INFO] Step 40_200: discriminator_loss=0.561489, adversarial_loss=1.033474
[INFO] Step 40_300: discriminator_loss=0.576912, adversarial_loss=0.990586
[INFO] Step 40_400: discriminator_loss=0.632197, adversarial_loss=0.692278
[INFO] Step 40_500: discriminator_loss=0.604369, adversarial_loss=0.695712
[INFO] Step 40_545: discriminator_loss=0.592075, adversarial_loss=1.